# Transfer learning en Pytorch

El concepto de transfer learning es un método donde el modelo que ya fue entrenado para una
aplicación es utilizado para otra aplicación. Por ejemplo, uno puede utilizar un modelo ya
*pre-entrenado* como punto de partida para desarrollar un modelo para una segunda aplicación.

En computer visión, esto es algo común que se utiliza y que ha demostrado una gran ventaja.

## Transferir aprendizaje con datos de imágenes

Es común realizar el aprendizaje por transferencia con problemas de modelado predictivo que utilizan datos de imagen como entrada.

Esta puede ser una tarea de predicción que toma fotografías o datos de video como entrada.

Para este tipo de problemas, es común utilizar un modelo de aprendizaje profundo previamente entrenado para una tarea de clasificación de
imágenes grande y desafiante, como la competencia de clasificación de fotografías de la clase [ImageNet](https://www.image-net.org/) 1000.

<img src="https://cv.gluon.ai/_images/imagenet_banner.jpeg" alt="neural_network" style="width: 700px;" align="center"/>

Dentro de "transfer learning", se destacan varios modelos. Por ejemplo:
- Modelo Oxford VGG
- Modelo Microsoft ResNet

Una de las razones por las que el "transfer learning" es llamativo es porque se puede utilizar
cuando uno tiene un **número pequeño o limitado de data**. 

Si quieren ver más sobre otros modelos que se han hecho públicos y ya están entrenados, se pueden dirigir a [Model Zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo) en Github.

### Cómo funciona el transfer learning

Uno puede pensar sobre "transfer learning' como un problema de optimización, en donde se trata de
optimizar las **últimas capas** del modelo y reentrenarlas para la aplicación de uno.

1. Uno entrena un modelo con una gran cantidad de data.
2. Ya que el modelo es entrenado, éste se puede utilizar para crear una aplicación bien general que se puede aplicar a varios problemas.
3. Si uno quiere utilizar este modelo para algo especifico, uno entrena solamente la parte final de la red con la data que no tiene.

La siguiente imagen representa el concept de "transfer learning" a más detalle:

<img src="https://i0.wp.com/neptune.ai/wp-content/uploads/Transfer-learning-idea.jpg?resize=768%2C432&ssl=1" alt="neural_network" style="width: 700px;" align="center"/>

Más específico:

<img src="https://i0.wp.com/neptune.ai/wp-content/uploads/Transfer-learning-steps.png?w=921&ssl=1" alt="neural_network" style="width: 700px;" align="center"/>
<img src="https://i0.wp.com/neptune.ai/wp-content/uploads/Transfer-learning-steps-2.png?w=850&ssl=1" alt="neural_network" style="width: 700px;" align="center"/>

## Ejemplo de Transfer Learning

En el siguiente tutorial, trataremos de clasificar imágenes de *abejas* y *hormigas*. Utilizaremos la data de 120
imágenes por cada clase para entrenar el modelo, y luego validaremos los resultados con 75 imágenes de *abejas* y
*hormigas*. El concepto de este tutorial fue adquirido de acá: [Link](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

### Descargando la data

Lo primero que tendremos que hacer es descargar la data necesaria:

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from pathlib import Path

In [ ]:
# Data para transfer learning
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip .
    
# Creando directorio donde se bajará la data:
transfer_learning_directorio = Path(".").joinpath("transfer_learning_data").resolve()
transfer_learning_directorio.mkdir(exist_ok=True, parents=True)
# extract it in the current folder
!unzip hymenoptera_data.zip -d ./transfer_learning_data

### Normalización y transformación de la data

El siguiente paso es preparar la data para que se pueda usar en el modelo pre-entrenado:

In [ ]:
# Augmentación de data:
# Un metodo para entrenar modelos de Deep Learning con imágenes es modificar la "apariencia" de
# las imágenes que se utilizan para el entrenamiento. Por ejemplo, se acostumbra a
# rotar las imágenes, normalizar los datos de la imágenes, etc. para que el modelo pueda
# aprender más "razgos" de las imágenes y por ende desarrollar un mejor modelo.
# 
# 
# Para las imágenes de validación, solamente normalizamos las imágenes.
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

Estas transformaciónes son solamente para que las imágenes de entrenamiento
y validación tengan las dimensiones correctas para el modelo.

### Definición de los datasets

El siguiente paso es la definición de los datasets:

In [ ]:
# Folder en el cuál se encuentran los archivos
data_dir = 'transfer_learning_data/hymenoptera_data'

# Creamos un diccionario para el "training" y "validation"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

# Creamos un "DataLoader", el cual nos dará las imágenes cada vez que las pidamos
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

# Extraemos los tamaños de los datasets
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# Los nombres de las diferentes clases
class_names = image_datasets['train'].classes

# Por último, definimos el dispositivo sobre el cuál se ejecutará el entrenamiento
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Examinando la data

Ya que hemos definido los diferentes datasets para el **entrenamiento** y la **validación**,
podemos visualizar algunas de las imágnes como parte de nuestro EDA:

In [ ]:
def imshow(inp, title=None):
    """
    Función para visualizar las imágenes
    """
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Agarramos un cierto número de datos y extraemos
# las imágenes y sus respectivos etiquetas ("labels").
# Esta es parte de la data de "entrenamiento"
inputs, classes = next(iter(dataloaders['train']))

# Creamos una figura con varias imágenes y las visualizamos
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

### Entrenamiento del modelo

La siguiente parte vamos a entrenar el modelo con las imágenes que tenemos.
Primero definiremos ciertas funciones que se usarán a lo largo del tutorial:

#### Función de entrenamiento
Esta función es la función principal para entrenar un modelo:

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    """
    Función para entrenar un modelo.
    
    Parámetros
    -------------
    model :
        Modelo que se utiliza y se está entrenando.
    
    criterion :
        Criterio que se utiliza como el "loss function".
    
    optimizer :
        Optimizador para modificar las pesas del modelo.
    
    scheduler :
        Objeto que modifica la "velocidad de aprendizaje"
        luego de un cierto número de épocas.
    
    num_epochs : int
        Número total de épocas a utilizar para entrenar
        el modelo.
    """
    # Empieza el entrenamiento
    since = time.time()
    
    # Copiando las pesas actuales del modelo
    best_model_wts = copy.deepcopy(model.state_dict())
    
    # Inicializando el valor de la "precisión" (accuracy)
    best_acc = 0.0

    # Es en este "loop" en donde empezamos a entrenar el modelo
    # y el modelo empieza a aprender de la data dada.
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Cada época tiene 2 fases: entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Hace que el modelo entre en modo de "entrenamiento"
            else:
                model.eval()   # Hace que el modelo entre en modo de "validación"

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Es en esta parte donde se aplica el
                    # "backpropagation" y optimización de las pesas
                    # NOTA: Esto solamente sucede durante el **entrenamiento**
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Recolectamos ciertas estadísticas del modelo
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Copiamos el modelo y lo preparamos para iniciar de nuevo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Descargamos las pesas más óptimas del modelo
    model.load_state_dict(best_model_wts)
    return model

La siguiente función es para visualizar los resultados del modelo:

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            # Este paso manda estos objectos al "dispositivo", i.e. "cpu" o "gpu".
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # En este paso el modelo crea predicciones
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

#### Mejorando el modelo pre-entrenado

Es ahora en esta fase donde podemos utilizar un modelo que ya haya sido completamente
entrenado con data similar a la que tenemos.

Para este ejemplo, utilizaremos el modelo **ResNet18** y utilizaremos sus pesas al principio:

In [ ]:
# Inicializamos el modelo y extraemos el número total de "features"
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# En esta parte se define el "fully-connected" layer, la cual se utilizará
# para definir el número total de clases que queremos. En este caso,
# estamos interesaos en ``2`` clases, y por ellos definimos **2** con parte
# de la arquitectura del modelo.
# 
# Esto se puede generalizar a ``nn.Linear(num_ftrs, len(class_names))``,
# dependiendo del número total de clasificaciones que queremos.
model_ft.fc = nn.Linear(num_ftrs, 2)
# Mandamos el objecto al dispositivo
model_ft = model_ft.to(device)

# En este caso decidimos el criterio / "loss function" de nuestro modelo.
# Utilizaremos el ``CrossEntropyLoss`` que se utiliza frequentemente para
# modelos de clasificación.
criterion = nn.CrossEntropyLoss()

# Definimos el "optimizador" y miramos que todos los parámetros del modelo
# sean optimizados.
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Finalmente podemos decidir un factor, por el cuál la "velocidad de entrenamiento"
# se reducirá después de cierto número de épocas.
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

#### Entrenando y evaluando los modelos

La última fase de entrenamiendo es ... entrenar el modelo. Uno puede definir si quiere entrenar
este modelo en un **CPU** o un **GPU**:

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Ahora podemos visualizar algunas de las predicciones del modelo:

In [ ]:
visualize_model(model_ft)

#### Entranando de nuevo el modelo, pero con nuestra data.

Ahora que ya hemos entrenado nuestro modelo con un modelo pre-entrenado, ahora podemos
cambiar de estrategia y utilizar "transfer learning" para entrenar otro modelo con nuestros datos:

In [ ]:
# Ahora vamos a utilizar las pesas de este modelo y entrenarlo con nuestra data.
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Volvemos a definir la forma del "fully-connected" layer para que tengo una salida
# de *2* clases.
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

# Mandamos el objeto al "CPU" o "GPU"
model_conv = model_conv.to(device)

# Loss function / Criterio
criterion = nn.CrossEntropyLoss()

# En este caso los parámetros de la "última capa" son optimizados.
# En el ejemplo anterior, esto no fue así.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Finalmente podemos decidir un factor, por el cuál la "velocidad de entrenamiento"
# se reducirá después de cierto número de épocas.
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Una vez más entrenamos el modelo:

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Y visualizamos los resultados:

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()